In [ ]:
# Import sqlite3 module
import sqlite3

In [ ]:
# Chetna-Connect to the playtime database
db=sqlite3.connect("/Users/livestrong19/APAD/APAD_Project1/PickupSport.db")
cursor = db.cursor()

In [ ]:
# Betty-Connect to the playtime database
db=sqlite3.connect("/Users/peihsinho/sqlite/db/PickupSport.db")
cursor = db.cursor()

In [ ]:
# Drop Venues table
cursor.execute('''DROP TABLE Venues''')
db.commit()

In [ ]:
# Create Venues table
cursor.execute('''
    CREATE TABLE Venues
                        (VenueId INTEGER PRIMARY KEY AUTOINCREMENT, 
                        VenueName TEXT NOT NULL,
                        VenueZipcode TEXT NOT NULL,
                        VenueAddress TEXT NOT NULL,
                        VenueOpeningTime TEXT NOT NULL,
                        VenueClosingTime TEXT NOT NULL,
                        VenueSports TEXT,
                        CHECK (LENGTH(VenueZipcode)==5 AND LENGTH(VenueName)<=15 AND LENGTH(VenueAddress)<=30))
                        ''')
db.commit()

In [ ]:
# Drop Users table
cursor.execute('''DROP TABLE users''')
db.commit()

In [ ]:
# Create Users table
cursor.execute('''
    CREATE TABLE users(uid INTEGER PRIMARY KEY AUTOINCREMENT, 
                       uname TEXT,
                       uphone TEXT,
                       uemail TEXT,
                       upassword TEXT,
                       uZipcode TEXT,
                       admin TEXT)
''')
db.commit()

In [ ]:
# Drop Events table
cursor.execute('''DROP TABLE Events''')
db.commit()

In [ ]:
# Create Events table
cursor.execute('''
    CREATE TABLE Events
                        (EventId INTEGER PRIMARY KEY AUTOINCREMENT, 
                        EventName TEXT NOT NULL,
                        EventDate TEXT NOT NULL,
                        EventSport TEXT NOT NULL,
                        EventTeamSize INTEGER NOT NULL,
                        EventSpotsLeft INTEGER NOT NULL,
                        EventDesc TEXT,
                        VenueId INTEGER NOT NULL,
                        EventHost TEXT NOT NULL,
                        EventStartTime TEXT NOT NULL,
                        EventEndTime TEXT NOT NULL,
                        FOREIGN KEY (EventHost) references users(uid),
                        FOREIGN KEY (VenueId) REFERENCES Venues(VenueId))
''')
db.commit()

In [ ]:
# Drop User_Events table
cursor.execute('''DROP TABLE User_Events''')
db.commit()

In [ ]:
# Create User_Events table
cursor.execute('''
    CREATE TABLE User_Events
                        (EventId Integer,
                         uid Integer,
                         FOREIGN KEY (EventId) REFERENCES Events(EventId), 
                         FOREIGN KEY (uid) REFERENCES users(uid))
''')
db.commit()

In [ ]:
# Check if user is Admin or not
def checkAdmin(uid):
    cursor.execute('''SELECT COUNT(*) FROM users WHERE admin = ? AND uid= ?''',('True', uid))
    all = cursor.fetchone()
    #Return true if such user found with Admin rights in Users table    
    if all[0] > 0:
        return True
    else:
        return False

# Check fucntion checkAdmin with sample input values
# checkAdmin(1)

In [ ]:
# Add new user in the Users table
def addUser(userId, uname, uphone, uemail, upassword, uZipcode, admin):
    
    if checkAdmin(userId):
        value = [(uname, uphone, uemail, upassword, uZipcode, admin)]
        cursor.executemany('''INSERT INTO users(uname, uphone, uemail, upassword, uZipcode ,admin) 
                      VALUES(?, ?, ?, ?, ?, ?)''', value)
        db.commit()
    else:
        print("You are not a Admin")
        
# Check fucntion addUser with sample input values
addUser(1, "Kyle", 23456, "d@gmail", "Why", 78705, "False")
addUser(1, "Bob", 85786, "s@gmail", "Dude", 98690, "False")
addUser(2, "Mary", 85744, "m@gmail", "Life", 92290, "False")

In [ ]:
# Calculate 1 hour time slots between start time and end time
def CalculateTimeslots(startTime, endTime):
    import datetime
    # Convert user input into date format
    start = datetime.datetime.strptime(startTime,'%H%M')
    end = datetime.datetime.strptime(endTime, '%H%M')
    slots = []
    # Create variable to perform date operations i.e. calculate next time after one hour
    one_hour = datetime.timedelta(minutes=60)  
    # Create list of starting times of all time slots
    while start < end and end >= start + one_hour:
            slots.append(start.strftime("%H%M"))
            start += one_hour
    return slots

# Check fucntion CalculateTimeslots with sample input values
# CalculateTimeslots("0900", "1100")
# CalculateTimeslots("0900", "1100")

In [ ]:
# Add a new venue in the Venues tables
# Follow 24 hour format for Opening & Closing Time column values 
def addVenue(userId, VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueSports):
   
    if checkAdmin(userId):
        value = [(VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueSports)]
        cursor.executemany('''INSERT INTO Venues(VenueName, VenueZipcode, VenueAddress, VenueOpeningTime, VenueClosingTime, VenueSports) 
                      VALUES(?, ?, ?, ?, ?,?)''', value)
        db.commit()
    else:
        print("User is not authorised to add a new venue")


# Populate Venues table with initial list of 10 venues
addVenue(1, "Venue 1", "78705", "26th Street", "0900", "1800", "Football")
addVenue(1, "Venue 2", "78706", "27th Street", "0900", "1700", "Basketball")
addVenue(1, "Venue 3", "78707", "28th Street", "0900", "1600", "Tennis")
addVenue(1, "Venue 4", "78708", "29th Street", "0900", "1500", "Swimming")
addVenue(1, "Venue 5", "78709", "30th Street", "0900", "1400", "Cricket")
addVenue(1, "Venue 6", "78700", "34th Street", "1000", "1800", "Football")
addVenue(1, "Venue 7", "78701", "35th Street", "1100", "1800", "Basketball")
addVenue(1, "Venue 8", "78702", "36th Street", "1200", "1800", "Tennis")
addVenue(1, "Venue 9", "78703", "37th Street", "1300", "1800", "Swimming")
addVenue(1, "Venue 10", "78704", "38th Street", "1200", "1700", "Cricket")
addVenue(2, "Venue 11", "78710", "39th Street", "0900", "1600", "Football, Hockey")

In [ ]:
# Add new event in Event table
def CreateEvent(EventHost, VenueID, EventName, EventDate, EventSport, EventTeamSize, 
                EventDesc, EventStartTime, EventEndTime):
    
    EventSpotsLeft = EventTeamSize - 1
    value = [(EventName, EventDate, EventSport, EventTeamSize, 
                EventSpotsLeft, EventDesc, VenueId, EventHost, EventStartTime,EventEndTime)]
    cursor.executemany('''INSERT INTO Events(EventName,EventDate, EventSport, EventTeamSize, 
                EventSpotsLeft, EventDesc, VenueId, EventHost, EventStartTime, EventEndTime) 
                      VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', value)
    db.commit()
    
    
    update user_event

# Check fucntion CreateEvent with sample input values
CreateEvent(2,'wrtg','arg','argg','arg',5, 5,'argrg','argr', 'srthgt')

In [ ]:
# Display timeslot availability at a venue on a given date
# User entry for Date value to follow this format 'YYYY-MM-DD'

def SingleVenueAllTimeslotAvailability (VenueId, Date):
    cursor.execute('''SELECT VenueOpeningTime, VenueClosingTime FROM Venues WHERE VenueId = ?''',(VenueId,))
    vCheck = cursor.fetchall()
    VenueTimeSlots = CalculateTimeslots(vCheck[0][0], vCheck[0][1])
    # Create a dictionary of venue timeslots as key and default value as available
    dictOfVenues = dict.fromkeys(VenueTimeSlots)
    for k,v in dictOfVenues.items():
        dictOfVenues[k] = 'Available'
    cursor.execute('''SELECT EventStartTime,EventEndTime FROM Events WHERE VenueId = ? AND EventDate = ?''',(VenueId,Date))
    eCheck = cursor.fetchall()
    EventDict = dict(eCheck)
    # Create a list of Event time slots - storing only the starting time
    EventTimeSlots = []
    for key,value in EventDict.items():
        EventTimeSlots.append(CalculateTimeslots(key, value))
    # If Event timeslot found in Venue Dictionary then update value of key as booked 
    for x in EventTimeSlots:
        if x[0] in dictOfVenues:
            dictOfVenues[x[0]] = 'Booked'
    Available_Timeslots = {key:value for key,value in dictOfVenues.items() if dictOfVenues[key] == 'Available'}
    #Returns a dictionary of available timeslots
    return Available_Timeslots

# Sample function call for testing
SingleVenueAllTimeslotAvailability (1, '2019-03-02')

In [ ]:
# Return venue availability for a particular timeslot
def SingleVenueSingleTimeslotAvailability (VenueId, Date, Start_Time, End_Time):
    VenueDict = SingleVenueAllTimeslotAvailability(VenueId, Date)
    GivenTimeSlots = CalculateTimeslots(Start_Time, End_Time)
    Available = notAvailable = 0
    for x in GivenTimeSlots:
        if x in VenueDict:
            Available += 1
        else:
            notAvailable += 1
    if notAvailable >0:
        return False
    else:
        return True
  
# Sample values to test SingleVenueSingleTimeslotAvailability function
print(SingleVenueSingleTimeslotAvailability (1, '2019-03-02', '0900', '1100'))
print(SingleVenueSingleTimeslotAvailability (1, '2019-03-02', '1100', '1200'))

In [ ]:
# Display available venues for a given timeslot on a given date
# User entry for Date value to follow this format 'YYYY-MM-DD'
# User entry fot Starting and Ending timeslot values to follow 24 hour format

def SingleTimeslotAllVenueAvailability (StartTime, EndTime, Date):
    #Fetch all venues from Venues table and create a dictionary for capturing venue availability
    cursor.execute('''SELECT VenueId FROM Venues''')
    vList = cursor.fetchall()
    vSimpleList = []
    [vSimpleList.append(First) for Row in vList for First in Row]
    vDict = dict.fromkeys(vSimpleList)
    for k,v in vDict.items():
        vDict[k] = 'Available'
    #For each venue check availability of given timeslot
    for key in vDict:
        vDict[key] = SingleVenueSingleTimeslotAvailability(key, Date, StartTime, EndTime)
    print(vDict)
    # Return dictionary with key = VenueID and value = Availability (True/False)
    Available_Venues = {key:value for key,value in vDict.items() if vDict[key]}
    return Available_Venues

# Sample values to test SingleTimeslotAllVenueAvailability function
print(SingleTimeslotAllVenueAvailability ('0900', '1100','2019-03-02'))